

**Model Text Generation**

PIC: Amanda Eliora

In [ ]:
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Flatten, TimeDistributed, Dropout, LSTMCell, RNN, Bidirectional, Concatenate, Layer
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.utils import tf_utils
from tensorflow.keras import backend as K
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import unicodedata
import re
import numpy as np
import os
import time
import shutil

import pandas as pd
import numpy as np
import re
import string, os 
tf.__version__

'2.7.0'

In [ ]:
# inisiliasi corpus
file = pd.read_csv('Twitter_Emotion_Dataset.csv')
corpus = [row for row in file['tweet']]

# contoh isi corpus
corpus[30:40]

['Foto saya di instagram masih ada cuma lupa password instanya jadi ngga bisa di buka !! ini pasti salah pemerintah dan instagram kenapa sampai pakai password segala !! Tuh sampai lupa khan gw #begoloe ',
 'Drama kmarin sore : seharian puasa trus pas lg mandi kaget bcz kedatangan tamu tak diundang pas bgt woi pas lagi adzan maghrib trus jd galau dong seharian itu dianggap sah puasa apa enggak-_-',
 'Sungguh mencintaimu berat sekali. Berpisah denganmu apalagi. #HadiahPerasaan - [USERNAME] Selalu bikin baper tp sukses melupakan laper. [URL]',
 '4. pas uda dikirim kembali, si kurir telp ngeluh dan mager buat nganter ke alamat yg bener. malah minta ketemuan di titik yg dia tau itupun dia gamau susah (no puter balik, hrs searah sama arah kendaraan dia jalan)',
 'Pada akhirnya kita akan melewati semua ini, semua yang dulu kita bayangkan betapa sulitnya. Kita lalui juga. Di akhir kita hanya bisa belajar, belajar, dan belajar. Memetik semua hikmah yang bisa kita petik. Mempersiapkan untuk kede

In [ ]:
# Preprocessing data
# membuat semua huruf lowercase
# usernamd dan url sudah di mask dalam dataset yang dipakai 

def preprocess(data):
  output = []
  for line in data:
    line.lower()
    output.append(line)
  return output

In [ ]:
# Tokenisasi

tokenizer = Tokenizer()
tokenized_corpus = tokenizer.fit_on_texts(corpus)
tokenized_corpus = tokenizer.texts_to_sequences(corpus)

In [ ]:
# Membagi data menjadi input dan output variables

dataX = []
dataY = []
for line in tokenized_corpus:
  token_list = line
  for i in range(0, len(token_list)):
    seg_in = token_list[:i+1]
    seg_out = token_list[i+1:]
    dataX.append(seg_in)
    dataY.append(seg_out)
n_patterns = len (dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  127167


In [ ]:
# padding data input
target_data = tf.keras.preprocessing.sequence.pad_sequences(dataX, padding="post")

# reshape X menjadi [samples, time steps, features]
target_data = target_data.reshape((target_data.shape[0], target_data.shape[1], 1))

In [ ]:
# parameter model LSTM

pd.set_option('display.max_colwidth', -1)
BATCH_SIZE = 200
embedding_dim = 100
units = 200

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [ ]:
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
print(vocab_size)

19443


In [ ]:
# Vectorisasi data dengan one-hot encoding

seq_length = 100
from tensorflow.keras.utils import to_categorical

sequences = np.array(sequences)
y = to_categorical(np.array(dataY))

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


NameError: ignored

In [ ]:
# Membangun model LSTM

model = keras.Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['sparse_categorical_accuracy'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, None, 100)         1944300   
                                                                 
 lstm_11 (LSTM)              (None, 100)               80400     
                                                                 
 dense_16 (Dense)            (None, 100)               10100     
                                                                 
 dense_17 (Dense)            (None, 19443)             1963743   
                                                                 
Total params: 3,998,543
Trainable params: 3,998,543
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(dataX, dataY, batch_size=256, epochs=5)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


ValueError: ignored